In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from mlxtend.frequent_patterns import apriori, association_rules 

# Basket Analysis/Association Rule Mining of Visits

In [3]:
df_raw = pd.read_excel("Overall_Raw_Adj.xls")

In [4]:
df_raw

,Insurance,Patient_Id,Date,Procedure,Units,Charge_Resp,Charge,Contractual_Adj,Charge Balance
0,Absolute Total Care Medicaid,72181,1/2/2020,90460,1,P,22.0,-1.84,0.0
1,Absolute Total Care Medicaid,72181,1/2/2020,90461,2,P,40.0,-16.24,0.0
2,Absolute Total Care Medicaid,72181,1/2/2020,99391,1,P,125.0,-28.41,0.0
3,Absolute Total Care Medicaid,72508,1/8/2020,99381,1,P,151.0,-43.87,0.0
4,Absolute Total Care Medicaid,72307,1/10/2020,90460,1,P,22.0,-1.84,0.0
...,...,...,...,...,...,...,...,...,...
36516,Valenz Tucker McClancy,69779,10/18/2021,90471,1,P,25.0,0.00,0.0
36517,Valenz Tucker McClancy,69779,10/18/2021,90686,1,P,30.0,0.00,0.0
36518,Valenz Tucker McClancy,69779,10/18/2021,99393,1,P,160.0,0.00,0.0
36519,WellCare Health Medicaid,72640,4/20/2020,99381,1,P,151.0,-67.95,0.0


### Add Visit ID column

In [13]:
patients = df_raw['Patient_Id'].apply(str)
dates = df_raw['Date'].apply(str)
visit_id = dict()
num = 1
col = []
for patient, date in zip(patients, dates):
    temp = patient + date
    visit_id[temp] = visit_id.get(temp,num)
    col.append(visit_id.get(temp))
    temp = ''
    num+=1
df_raw['Visit_Id'] = col

In [23]:
df_raw

,Insurance,Patient_Id,Date,Procedure,Units,Charge_Resp,Charge,Contractual_Adj,Charge Balance,Visit_Id
0,Absolute Total Care Medicaid,72181,1/2/2020,90460,1,P,22.0,-1.84,0.0,1
1,Absolute Total Care Medicaid,72181,1/2/2020,90461,2,P,40.0,-16.24,0.0,1
2,Absolute Total Care Medicaid,72181,1/2/2020,99391,1,P,125.0,-28.41,0.0,1
3,Absolute Total Care Medicaid,72508,1/8/2020,99381,1,P,151.0,-43.87,0.0,4
4,Absolute Total Care Medicaid,72307,1/10/2020,90460,1,P,22.0,-1.84,0.0,5
...,...,...,...,...,...,...,...,...,...,...
36516,Valenz Tucker McClancy,69779,10/18/2021,90471,1,P,25.0,0.00,0.0,36517
36517,Valenz Tucker McClancy,69779,10/18/2021,90686,1,P,30.0,0.00,0.0,36517
36518,Valenz Tucker McClancy,69779,10/18/2021,99393,1,P,160.0,0.00,0.0,36517
36519,WellCare Health Medicaid,72640,4/20/2020,99381,1,P,151.0,-67.95,0.0,36520


### Group by Visit Id

In [29]:
df_raw.groupby(by=['Visit_Id', 'Procedure'], as_index=False)

In [33]:
df = df_raw.groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')

In [51]:
df_mb = df.applymap(lambda x: False if x<=0 else True)

### Total Association Rule Mining

In [67]:
freq_itemsets = apriori(df_mb, min_support=0.05, use_colnames=True)
freq_itemsets

,support,itemsets
0,0.092534,(87880)
1,0.131674,(90460)
2,0.072421,(90461)
3,0.187364,(90471)
4,0.098031,(90472)
5,0.051160,(90670)
6,0.064810,(90686)
7,0.334622,(99213)
8,0.129681,(99214)
9,0.118748,(99391)


In [71]:
rules = association_rules(freq_itemsets, metric='lift', min_threshold=1)
rules.sort_values(by=['confidence','support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(90472),(90471),0.098031,0.187364,0.098031,1.000000,5.337202,0.079663,inf
8,(90686),(90471),0.064810,0.187364,0.064810,1.000000,5.337202,0.052667,inf
6,(90670),(90471),0.051160,0.187364,0.051160,1.000000,5.337202,0.041574,inf
13,(90670),(90472),0.051160,0.098031,0.051160,1.000000,10.200863,0.046144,inf
14,"(90472, 90670)",(90471),0.051160,0.187364,0.051160,1.000000,5.337202,0.041574,inf
16,"(90670, 90471)",(90472),0.051160,0.098031,0.051160,1.000000,10.200863,0.046144,inf
18,(90670),"(90472, 90471)",0.051160,0.098031,0.051160,1.000000,10.200863,0.046144,inf
3,(90461),(90460),0.072421,0.131674,0.072300,0.998332,7.581827,0.062764,520.561247
0,(87880),(99213),0.092534,0.334622,0.063119,0.682115,2.038465,0.032155,2.093140
2,(90460),(90461),0.131674,0.072421,0.072300,0.549083,7.581827,0.062764,2.057093


### Association Rule Mining for Most frequent 6 Insurances (31,295 data points or 85.7% of total points)

In [75]:
df_raw.Insurance.value_counts()

{'BCBS - PPO Plans': 9772,
 'Molina Healthcare Medicaid': 9327,
 'BCBS SC SHP': 5749,
 'Healthy Blue Medicaid': 2410,
 'United Healthcare': 2252,
 'Cigna': 1785,
 'Absolute Total Care Medicaid': 923,
 'Aetna': 840,
 'No Primary Insurance Plan': 521,
 'Medicaid of SC': 470,
 'UMR': 459,
 'BlueChoice Commercial Plan': 345,
 'Medcost': 341,
 'Tricare SOUTH Region': 267,
 'BCBS Federal Employee Program': 227,
 'National Gen Accident-Health': 69,
 'Cigna PPO via Healthgram': 64,
 'Medi-Share': 59,
 'Trustmark Health': 55,
 'Allegiance via Cigna': 49,
 'Liberty Healthshare': 49,
 'BCBS NC SHP': 45,
 'Ambetter': 38,
 'Golden Rule': 38,
 'Humana': 36,
 'GEHA': 34,
 'PHCS via Aspen Insurance': 28,
 'All Savers': 26,
 'First Choice Medicaid': 24,
 'Disability Determination Payor': 19,
 'Corporate Benefit Services  In': 19,
 'Evolution Healthcare': 18,
 'Key Benefit Administrators': 18,
 'Meritain Health - Aetna': 15,
 'Allied Benefit Systems': 15,
 'Trustmark Small Business': 13,
 'Loomis (Healt

In [91]:
df_top_ins = df_raw.groupby('Insurance').filter(lambda x: len(x)>1000)

#### BCBS - PPO Plans

In [97]:
df_bcbsppo = df_top_ins[df_top_ins['Insurance']=='BCBS - PPO Plans'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_bcbsppo = df_bcbsppo.applymap(lambda x: False if x<=0 else True)
freq_itemsets_bcbsppo = apriori(df_bcbsppo, min_support=0.05, use_colnames=True)
freq_itemsets_bcbsppo

,support,itemsets
0,0.093369,(87880)
1,0.350133,(90471)
2,0.183820,(90472)
3,0.071088,(90474)
4,0.052255,(90633)
...,...,...
144,0.070822,"(90723, 90471, 90472, 90474, 90680, 99391)"
145,0.070822,"(90723, 90471, 90472, 90670, 90680, 99391)"
146,0.070822,"(90723, 90471, 90474, 90670, 90680, 99391)"
147,0.070822,"(90723, 90472, 90474, 90670, 90680, 99391)"


In [100]:
rules_bcbsppo = association_rules(freq_itemsets_bcbsppo, metric='lift', min_threshold=1)
rules_bcbsppo.sort_values(by=['confidence','support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(90472),(90471),0.183820,0.350133,0.183820,1.000000,2.856061,0.119458,inf
16,(90686),(90471),0.119629,0.350133,0.119629,1.000000,2.856061,0.077743,inf
12,(90670),(90471),0.095225,0.350133,0.095225,1.000000,2.856061,0.061884,inf
31,(90670),(90472),0.095225,0.183820,0.095225,1.000000,5.440115,0.077721,inf
72,"(90472, 90670)",(90471),0.095225,0.350133,0.095225,1.000000,2.856061,0.061884,inf
...,...,...,...,...,...,...,...,...,...
70,(90471),"(90472, 90647)",0.350133,0.069231,0.069231,0.197727,2.856061,0.044991,1.160166
101,(90471),"(90472, 99392)",0.350133,0.067374,0.067374,0.192424,2.856061,0.043784,1.154846
21,(90471),(90734),0.350133,0.052785,0.052785,0.150758,2.856061,0.034303,1.115365
7,(90471),(90633),0.350133,0.052255,0.052255,0.149242,2.856061,0.033959,1.114002


#### Molina Healthcare Medicaid

In [101]:
df_mhm = df_top_ins[df_top_ins['Insurance']=='Molina Healthcare Medicaid'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_mhm = df_mhm.applymap(lambda x: False if x<=0 else True)
freq_itemsets_mhm = apriori(df_mhm, min_support=0.05, use_colnames=True)
freq_itemsets_mhm

,support,itemsets
0,0.126535,(87880)
1,0.240983,(90460)
2,0.137649,(90461)
3,0.061805,(99212)
4,0.406902,(99213)
5,0.158510,(99214)
6,0.063170,(99391)
7,0.098850,(99392)
8,0.120101,(99393)
9,0.085202,(99394)


In [102]:
rules_mhm = association_rules(freq_itemsets_mhm, metric='lift', min_threshold=1)
rules_mhm.sort_values(by=['confidence','support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(90461),(90460),0.137649,0.240983,0.137649,1.000000,4.149676,0.104478,inf
9,"(99392, 90461)",(90460),0.055176,0.240983,0.055176,1.000000,4.149676,0.041880,inf
4,(99392),(90460),0.098850,0.240983,0.081107,0.820513,3.404863,0.057286,4.228811
8,"(99392, 90460)",(90461),0.081107,0.137649,0.055176,0.680288,4.942209,0.044012,2.697279
0,(87880),(99213),0.126535,0.406902,0.085592,0.676425,1.662379,0.034104,1.832955
2,(90460),(90461),0.240983,0.137649,0.137649,0.571197,4.149676,0.104478,2.011068
6,(99392),(90461),0.098850,0.137649,0.055176,0.558185,4.055146,0.041570,1.951840
11,(99392),"(90460, 90461)",0.098850,0.137649,0.055176,0.558185,4.055146,0.041570,1.951840
7,(90461),(99392),0.137649,0.098850,0.055176,0.400850,4.055146,0.041570,1.504048
10,"(90460, 90461)",(99392),0.137649,0.098850,0.055176,0.400850,4.055146,0.041570,1.504048


#### BCBS SC SHP

In [103]:
df_bcbssc = df_top_ins[df_top_ins['Insurance']=='BCBS SC SHP'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_bcbssc = df_bcbssc.applymap(lambda x: False if x<=0 else True)
freq_itemsets_bcbssc = apriori(df_bcbssc, min_support=0.05, use_colnames=True)
freq_itemsets_bcbssc

,support,itemsets
0,0.081967,(87880)
1,0.353322,(90471)
2,0.172563,(90472)
3,0.067299,(90474)
4,0.066005,(90647)
...,...,...
140,0.067299,"(90723, 90471, 90472, 90474, 90680, 99391)"
141,0.067299,"(90723, 90471, 90472, 90670, 90680, 99391)"
142,0.067299,"(90723, 90471, 90474, 90670, 90680, 99391)"
143,0.067299,"(90723, 90472, 90474, 90670, 90680, 99391)"


In [104]:
rules_bcbssc = association_rules(freq_itemsets_bcbssc, metric='lift', min_threshold=1)
rules_bcbssc.sort_values(by=['confidence','support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(90472),(90471),0.172563,0.353322,0.172563,1.000000,2.830281,0.111592,inf
12,(90686),(90471),0.134599,0.353322,0.134599,1.000000,2.830281,0.087042,inf
8,(90670),(90471),0.087575,0.353322,0.087575,1.000000,2.830281,0.056633,inf
27,(90670),(90472),0.087575,0.172563,0.087575,1.000000,5.795000,0.072463,inf
68,"(90472, 90670)",(90471),0.087575,0.353322,0.087575,1.000000,2.830281,0.056633,inf
...,...,...,...,...,...,...,...,...,...
7,(90471),(90647),0.353322,0.066005,0.066005,0.186813,2.830281,0.042684,1.148561
67,(90471),"(90472, 90647)",0.353322,0.066005,0.066005,0.186813,2.830281,0.042684,1.148561
1,(99213),(87880),0.310181,0.081967,0.056946,0.183588,2.239777,0.031521,1.124473
97,(90471),"(90472, 99392)",0.353322,0.063417,0.063417,0.179487,2.830281,0.041010,1.141461


#### Healthy Blue Medicaid

In [105]:
df_hbm = df_top_ins[df_top_ins['Insurance']=='Healthy Blue Medicaid'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_hbm = df_hbm.applymap(lambda x: False if x<=0 else True)
freq_itemsets_hbm = apriori(df_hbm, min_support=0.05, use_colnames=True)
freq_itemsets_hbm

,support,itemsets
0,0.415065,(90460)
1,0.235204,(90461)
2,0.055342,(96110)
3,0.292083,(99213)
4,0.081476,(99214)
5,0.063028,(99381)
6,0.299769,(99391)
7,0.162952,(99392)
8,0.235204,"(90460, 90461)"
9,0.053036,"(90460, 96110)"


In [106]:
rules_hbm = association_rules(freq_itemsets_hbm, metric='lift', min_threshold=1)
rules_hbm.sort_values(by=['confidence','support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(90461),(90460),0.235204,0.415065,0.235204,1.000000,2.409259,0.137579,inf
16,"(90461, 99391)",(90460),0.149116,0.415065,0.149116,1.000000,2.409259,0.087223,inf
21,"(99392, 90461)",(90460),0.076095,0.415065,0.076095,1.000000,2.409259,0.044511,inf
28,"(90460, 96110)",(99392),0.053036,0.162952,0.052267,0.985507,6.047853,0.043625,57.756341
13,(96110),(99392),0.055342,0.162952,0.053805,0.972222,5.966326,0.044787,30.133743
6,(99392),(90460),0.162952,0.415065,0.158340,0.971698,2.341073,0.090704,20.667692
27,"(99392, 96110)",(90460),0.053805,0.415065,0.052267,0.971429,2.340423,0.029935,20.472713
3,(96110),(90460),0.055342,0.415065,0.053036,0.958333,2.308873,0.030066,14.038432
31,(96110),"(99392, 90460)",0.055342,0.158340,0.052267,0.944444,5.964671,0.043505,15.149885
15,"(90460, 99391)",(90461),0.196772,0.235204,0.149116,0.757812,3.221941,0.102835,3.157869


#### United Healthcare

In [108]:
df_uh = df_top_ins[df_top_ins['Insurance']=='United Healthcare'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_uh = df_uh.applymap(lambda x: False if x<=0 else True)
freq_itemsets_uh = apriori(df_uh, min_support=0.05, use_colnames=True)
freq_itemsets_uh

,support,itemsets
0,0.060742,(87880)
1,0.354331,(90471)
2,0.174353,(90472)
3,0.059618,(90474)
4,0.061867,(90633)
...,...,...
145,0.058493,"(90723, 90471, 90472, 90474, 90680, 99391)"
146,0.058493,"(90723, 90471, 90472, 90670, 90680, 99391)"
147,0.058493,"(90723, 90471, 90474, 90670, 90680, 99391)"
148,0.058493,"(90723, 90472, 90474, 90670, 90680, 99391)"


In [109]:
rules_uh = association_rules(freq_itemsets_uh, metric='lift', min_threshold=1)
rules_uh.sort_values(by=['confidence','support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(90472),(90471),0.174353,0.354331,0.174353,1.000000,2.822222,0.112575,inf
12,(90686),(90471),0.121485,0.354331,0.121485,1.000000,2.822222,0.078439,inf
8,(90670),(90471),0.087739,0.354331,0.087739,1.000000,2.822222,0.056650,inf
27,(90670),(90472),0.087739,0.174353,0.087739,1.000000,5.735484,0.072441,inf
72,"(90472, 90670)",(90471),0.087739,0.354331,0.087739,1.000000,2.822222,0.056650,inf
...,...,...,...,...,...,...,...,...,...
1781,(90471),"(90723, 90474, 90670, 90680, 99391)",0.354331,0.058493,0.058493,0.165079,2.822222,0.037767,1.127661
1968,(90471),"(90723, 90472, 90474, 90670, 90680, 99391)",0.354331,0.058493,0.058493,0.165079,2.822222,0.037767,1.127661
17,(90471),(96110),0.354331,0.057368,0.056243,0.158730,2.766885,0.035916,1.120487
131,(90471),"(99392, 90633)",0.354331,0.055118,0.055118,0.155556,2.822222,0.035588,1.118939


#### Cigna

In [111]:
df_c = df_top_ins[df_top_ins['Insurance']=='Cigna'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_c = df_c.applymap(lambda x: False if x<=0 else True)
freq_itemsets_c = apriori(df_c, min_support=0.05, use_colnames=True)
freq_itemsets_c

,support,itemsets
0,0.090014,(87880)
1,0.340366,(90471)
2,0.177215,(90472)
3,0.066104,(90474)
4,0.057665,(90633)
...,...,...
148,0.064698,"(90723, 90471, 90472, 90474, 90680, 99391)"
149,0.064698,"(90723, 90471, 90472, 90670, 90680, 99391)"
150,0.064698,"(90723, 90471, 90474, 90670, 90680, 99391)"
151,0.064698,"(90723, 90472, 90474, 90670, 90680, 99391)"


In [112]:
rules_c = association_rules(freq_itemsets_c, metric='lift', min_threshold=1)
rules_c.sort_values(by=['confidence','support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(90472),(90471),0.177215,0.340366,0.177215,1.000000,2.938017,0.116897,inf
14,(90686),(90471),0.115331,0.340366,0.115331,1.000000,2.938017,0.076076,inf
10,(90670),(90471),0.098453,0.340366,0.098453,1.000000,2.938017,0.064943,inf
31,(90670),(90472),0.098453,0.177215,0.098453,1.000000,5.642857,0.081006,inf
76,"(90472, 90670)",(90471),0.098453,0.340366,0.098453,1.000000,2.938017,0.064943,inf
...,...,...,...,...,...,...,...,...,...
7,(90471),(90633),0.340366,0.057665,0.057665,0.169421,2.938017,0.038038,1.134552
19,(90471),(90716),0.340366,0.052039,0.052039,0.152893,2.938017,0.034327,1.119056
99,(90471),"(90472, 90716)",0.340366,0.052039,0.052039,0.152893,2.938017,0.034327,1.119056
17,(90471),(90707),0.340366,0.050633,0.050633,0.148760,2.938017,0.033399,1.115276
